In [129]:
import pennylane as qml
import pennylane.numpy as np

In [187]:
def U_psi(theta):
    """
    Quantum function that generates |psi>, Zenda's state wants to send to Reece.

    Args:
        theta (float): Parameter that generates the state.

    """
    qml.Hadamard(wires = 0)
    qml.CRX(theta, wires = [0,1])
    qml.CRZ(theta, wires = [0,1])

In [280]:
def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """

    O = qml.matrix((qml.RX(beta,wires=0)  @ qml.RX(beta,wires=1))
                          @ (qml.RZ(alpha,wires=0) @ qml.RZ(alpha,wires=1)))
    
    psi_bra = np.array([1,0,0,0]) 
    psi_ket = psi_bra.T

    is_unsafe = False

    thetas = np.linspace(0,2*np.pi,num=100)
    for theta in thetas:
        U = qml.matrix(U_psi,wire_order=[0,1])(theta)
        Udg = qml.matrix(qml.adjoint(U_psi),wire_order=[0,1])(theta)
        exp_val = psi_bra @ Udg @ O @ U @ psi_ket

        if np.absolute(exp_val)**2 >= (1 - epsilon):
            is_unsafe = True
    

    return is_unsafe


In [283]:
test1 = [0.1, 0.2, 0.3] # true
test2 = [1.1, 1.2, 0.3] # false
test3 = [1.1, 1.2, 0.4] # true
test4 = [0.5, 1.9, 0.7] # true

is_unsafe(*test2)

False

In [291]:
# Step 1: initialize a device
dev = qml.device("default.qubit",wires=1)

# Step 2: Add a decorator below
@qml.qnode(dev)
def simple_circuit(angle):

    """
    In this function:
        * Rotate the qubit around the y-axis by angle
        * Measure the expectation value of the Pauli X observable

    Args:
        angle (float): how much to rotate a state around the y-axis

    Returns:
        Union[tensor, float]: The expectation value of the Pauli X observable
    """
    

    qml.RY(angle,wires=0)

    return qml.expval(qml.PauliX(wires=0))

In [292]:
simple_circuit(0.1)

tensor(0.09983342, requires_grad=True)